Exercise: Volume and Claims
------------------------

First we need a `PersistentVolume` to which the `Claims` are headed.

In the example below, the `hostPath` is made available as a volume:

    kind: PersistentVolume
    apiVersion: v1
    metadata:
      name: data-volume
      labels:
        type: local
    spec:
      storageClassName: manual
      capacity:
        storage: 10Gi
      accessModes:
        - ReadWriteMany
      hostPath:
        path: "/data"


This is followed by the `Claims`:

    kind: PersistentVolumeClaim
    apiVersion: v1
    metadata:
      name: data-claim
    spec:
      storageClassName: manual
      accessModes:
        - ReadWriteMany
      resources:
        requests:
          storage: 10Gi

The pod itself points to the `Claims`:

      volumes:
      - name: web-storage
        persistentVolumeClaim:
        claimName: data-claim    

The pod, however, must first release its directories using `volumeMounts`:

    volumeMounts:
    - mountPath: "/usr/local/apache2/htdocs"
      subPath: htdocs       
      name: "web-storage"

The complete example of a web server storing its files on the host looks like this:


In [ ]:
! cat 09-5-Volume/apache.yaml 

In [ ]:
! kubectl apply -f 09-5-Volume/apache.yaml

In order to display more than one blank page, an `index.html` file must now be created in the `/data/htdocs` directory on the **in the VM**.

The content can be created as follows:

In [ ]:
%%bash
cat <<%EOF% >/data/htdocs/index.html
    <html>
        <body>
            <h1>Hello from Container</h1>
        </body>
    </html>
%EOF%
ls -l /data/htdocs

In [ ]:
! kubectl get service/apache

Since we don't have a load balancer, we have to prepare the IP of the cluster and the mapped port (port-based-routing) as a URL using a small shell script

Opening this URL in a new tab will show `Hello Host`.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service apache -o=jsonpath='{ .spec.ports[0].nodePort }')

- - -

Clean up.

The file `lernkube/data/htdocs/index.html` is retained because it is stored on `PersistentVolume`.

In [ ]:
! kubectl delete -f 09-5-Volume/apache.yaml